This is a program which is in testing phase. We will use a linear regression ML model to give an SOC number for the flights.

In [ ]:
# Import the querying module
from flight_querying import query_flights
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Set up and retrieve the data from the database.
db_connect = query_flights()

In [ ]:
test_id = 4929

df = db_connect.connect_flight_for_ml_data_prescription(test_id)

df = df[df["activity"] != "NA"]

In [ ]:
df.head()


In [ ]:
df.tail(10)

In [ ]:
# Make the data dictionary holder
data_dict = {
    "Time Delta": [],
    "SOC Delta": [],
    "Activity": [],
    "Average Power": [],
    "Id": []
}

In [ ]:
# Set index and initial exercise
start_index = 0
current_exercise = df.iloc[0, df.columns.get_loc('activity')]

# iterate over all the rows
for index, row in df.iterrows():

    # Initalize new exercise
    new_exercise = row["activity"]

    # See if the exercise changes
    if current_exercise != new_exercise or index == len(df):

        # Get SOC for the exercise and store the change from max to min.
        soc_range = df.iloc[start_index:index-1, df.columns.get_loc('soc')]
        soc_max, soc_min = np.max(soc_range), np.min(soc_range)
        data_dict["SOC Delta"].append(soc_max - soc_min)

        print(f"SOC Max: {soc_max}, SOC Min: {soc_min}")

        # Get Time for the exercise and store the change from max to min.
        time_range = df.iloc[start_index:index-1, df.columns.get_loc('time')]
        time_start, time_end = np.min(time_range), np.max(time_range)
        data_dict["Time Delta"].append(round(time_end - time_start, 2))

        # Get the Average Power
        power_average = np.mean(df.iloc[start_index:index-1, df.columns.get_loc('power')])
        data_dict["Average Power"].append(round(power_average, 2))

        # Set the id and activity
        data_dict["Id"].append(row["fw_flight_id"])
        data_dict["Activity"].append(current_exercise)

        # Reset the index and exercise to the one 
        start_index = index
        current_exercise = new_exercise
    
    if index == len(df):
        break

In [ ]:
final_df = pd.DataFrame(data_dict)
print(final_df)

In [ ]:
# Get the current exercise
current_exercise = df.iloc[0, 2]
max_soc = 0
min_soc = 101
min_power = 100000
max_power = -1000
min_time = 100
max_time = 0

# iterate over all the rows
for index, row in df.iterrows():

    new_exercise = row["activity"]
    soc = row["soc"]
    power = row["power"]
    time = row["time"]

    if current_exercise == new_exercise:
        if soc >= max_soc:
            max_soc = soc
        if soc <= min_soc:
            min_soc = soc
        if power >= max_power:
            max_power = power
        if power <= min_power:
            min_power = power
        if time >= max_time:
            max_time = time
        if time <= min_time:
            min_time = time
        id = row["fw_flight_id"]
    else:
        data_dict["Time Delta"].append(round(max_time - min_time, 2))
        data_dict["SOC Delta"].append(max_soc - min_soc)
        data_dict["Activity"].append(current_exercise)
        data_dict["Average Power"].append(max_power - min_power)
        data_dict["Id"].append(id)

        current_exercise = new_exercise
        max_soc, min_soc = soc, soc
        max_power, min_power = power, power
        max_time, min_time = time, time

print(data_dict)


In [ ]:
final_df = pd.DataFrame(data_dict)
print(final_df)

In [ ]:
a = np.arange(6)
print(a)
for x in np.nditer(a):
    print(x, end=', ')

In [ ]:
# Flight IDs to include
flight_ids = [4620, 4929, 4940, 5019, 5021, 5034]

# Fetch data for specified flight IDs
data_frames = [db_connect.connect_flight_for_ml_data_prescription(flight_id) for flight_id in flight_ids]

# Concatenate data frames and shuffle the data
all_data = pd.concat(data_frames, axis=0).sample(frac=1, random_state=42)

# Split the data into train and test sets
train_data, test_data = train_test_split(all_data, test_size=0.5, random_state=42)

In [ ]:
print(f"Length train_x = {len(train_data)} \n Length train_y = {len(test_data)}")

One-Hot-Encoding of the Operations columns

In [ ]:
# ONE-HOT ENCODE
# https://stackabuse.com/one-hot-encoding-in-python-with-pandas-and-scikit-learn/
def one_hot(df, col, pre):
  encoded = pd.get_dummies(df[col], prefix=pre)
  for column in encoded:
    encoded = encoded.rename(columns={column: col + "_" + column})
  encoded['time'] = df['time']
  encoded["id"] = df["id"]
  return encoded

In [ ]:
# Encode Train data
train_encoded = one_hot(train_data, "exercise", 'is')
final_train_x = pd.merge(train_data, train_encoded, on=["time", "id"])
final_train_y = final_train_x["soc"].to_numpy()
final_train_x = final_train_x.drop(columns=["time", "id", "exercise", "soc"])

# Encode Test data
test_encoded = one_hot(test_data, "exercise", 'is')
final_test_x = pd.merge(test_data, test_encoded, on=["time","id"])
final_test_y = final_test_x["soc"].to_numpy()
final_test_x = final_test_x.drop(columns=["time", "id", "exercise", "soc"])

In [ ]:
final_train_x

In [ ]:
print(f"Length test_encoded = {len(test_encoded)} \n Length train_encoded = {len(train_encoded)}")

In [ ]:
print(f"Length train_x = {len(final_train_x)} \n Length train_y = {len(final_train_y)}")

In [ ]:
print(f"Length test_x = {len(final_test_x)} \n Length test_y = {len(final_test_y)}")

Machine Learning Model Implementation

In [ ]:
# import sklearn
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, classification_report

# Set model
regression_model = LinearRegression()

# Fit model
regression_model.fit(final_train_x, final_train_y)

In [ ]:
# print model score
print(regression_model.score(final_test_x, final_test_y))

In [ ]:
coeff = pd.DataFrame(list(zip(regression_model.feature_names_in_, regression_model.coef_)), columns = ['Feature', 'Weight'])
coeff.sort_values('Weight')